# NNodely Documentation - Inference

Here are shown all the modalities by which you can make inference with the model.

In [1]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>-- nnodely_v1.5.0 --<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


## Simple inference

To make inference, just call the nnodely framework class by passing the dictionary of inputs. The model will execute a forward pass and return the outputs of the network.

It is mandatory to call the inference with at least 1 possible prediction window for each declared inputs. the framework will predict the maximum possible window given the data

In [2]:
## Model definition
x = Input('x')
F = Input('F')

next_x = Fir()(x.tw(0.5))+Fir()(F.last())

out = Output('next_x', next_x)

model = Modely()
model.addModel('model',[out])
model.neuralizeModel(0.05)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 0.05,
          'nnodely_version': '1.5.0',
          'ns': [10, 0],
          'ntot': 10,
          'num_parameters': 11},
 'Inputs': {'F': {'dim': 1, 'ns': [1, 0], 'ntot': 1, 'sw': [-1, 0]},
            'x': {'dim': 1, 'ns': [10, 0], 'ntot': 10, 'tw': [-0.5, 0]}},
 'Models': 'model',
 'Outputs': {'next_x': 'Add6'},
 'Parameters': {'PFir3W': {'dim': 1,
                           'tw': 0.5,
                           'values': [[0.7055824398994446],
                                      [0.6593199968338013],
                                      [0.19700944423675537],
                                      [0.9291446208953857],
                                      [0.6154297590255737],
                                      [0.8444268107414246],
                                      [0.39195936918258667],
                                      [0.10

In [3]:
## Inference
results = model(inputs={'F':[[9]],'x':[[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11]]})
print(results)

{'next_x': [34.583377838134766]}


In [4]:
results = model(inputs={'F':[[5],[4],[9]],'x':[[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12],[13]]})
print(results)

{'next_x': [31.785030364990234, 36.58085250854492, 45.57419204711914]}


## Inference with a sampled data

Using the option sampled=True the inference window is left to the user to define by passing every sampling input.

In [5]:
results = model(inputs={'F':[[5],[2]],'x':[[1,2,3,4,5,6,7,8,9,10],[12,13,14,15,16,17,18,19,20,21]]}, sampled=True)
print(results)

{'next_x': [33.1842041015625, []]}


## Closed Loop and Connect during Inference

During inference time you can define closed loop and connect relations between the variables.

In [6]:
input1 = Input('in1')
W = Parameter('W', sw=3, values=[[1], [2], [3]])
out = Output('out',Fir(W=W)(input1.sw(3)))

model = Modely(visualizer=TextVisualizer(), seed=42)
model.addModel('model', [out])
model.neuralizeModel()

result = model({'in1': [1, 2, 3]}, closed_loop={'in1':'out'})
print(result)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 1,
          'nnodely_version': '1.5.0',
          'ns': [3, 0],
          'ntot': 3,
          'num_parameters': 3},
 'Inputs': {'in1': {'dim': 1, 'ns': [3, 0], 'ntot': 3, 'sw': [-3, 0]}},
 'Models': 'model',
 'Outputs': {'out': 'Fir9'},
 'Parameters': {'W': {'dim': 1,
                      'init_values': [[1.0], [2.0], [3.0]],
                      'sw': 3,
                      'values': [[1.0], [2.0], [3.0]]}},
 'Relations': {'Fir9': ['Fir', ['SamplePart8'], 'W', None, 0],
               'SamplePart8': ['SamplePart', ['in1'], -1, [-3, 0]]}}
{'out': [14.0]}


In [7]:
input2 = Input('in2')
K = Parameter('K', sw=3, values=[[1], [2], [3]])
out2 = Output('out2',Fir(W=K)(input2.sw(3)))

model.addModel('model2', [out2])
model.neuralizeModel()

result = model(inputs={'in1': [1, 2, 3]}, connect={'in2':'out'})
print(result)

================================ nnodely Model =================================
{'Constants': {},
 'Functions': {},
 'Info': {'SampleTime': 1,
          'nnodely_version': '1.5.0',
          'ns': [3, 0],
          'ntot': 3,
          'num_parameters': 6},
 'Inputs': {'in1': {'dim': 1, 'ns': [3, 0], 'ntot': 3, 'sw': [-3, 0]},
            'in2': {'dim': 1, 'ns': [3, 0], 'ntot': 3, 'sw': [-3, 0]}},
 'Models': {'model': {'Constants': [],
                      'Functions': [],
                      'Inputs': ['in1'],
                      'Outputs': ['out'],
                      'Parameters': ['W'],
                      'Relations': ['SamplePart8', 'Fir9']},
            'model2': {'Constants': [],
                       'Functions': [],
                       'Inputs': ['in2'],
                       'Outputs': ['out2'],
                       'Parameters': ['K'],
                       'Relations': ['SamplePart11', 'Fir12']}},
 'Outputs': {'out': 'Fir9', 'out2': 'Fir12'},
 'Parameters

## Recurrent inference with prediction samples

to do multiple recurrent inference with the state variables just decide the number of samples to predict using the attribute 'prediction_samples'.

it can also be 'auto' meaning that the recurrent inference will continue until there are inputs available.

In [8]:
result = model({'in1': [1, 2, 3, 4, 5]},closed_loop={'in1':'out'}, prediction_samples=3)
print(result)

{'out2': [0.0, 0.0, 0.0], 'out': [14.0, 50.0, 181.0]}


## Recurrent inference with number of samples

By specifying the number of samples when doing inference with recurrent input variables

The number of samples will specify for how many steps the inference must go on. This can be done only when using recurrent input variables 

In [9]:
result = model({'in1': [1, 2, 3, 4, 5]},closed_loop={'in1':'out'}, prediction_samples=3, num_of_samples = 5)
print(result)

{'out2': [0.0, 0.0, 0.0, 0.0, 0.0], 'out': [14.0, 50.0, 181.0, 657.0, 5.0]}
